In [ ]:
# This processes raw data

# Import relevant libaries
%matplotlib qt5 

import mne
import os
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs, create_ecg_epochs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# DO THIS TO EACH RECORDING SEPARATELY

# set data folder path
eeg_path = os.path.join(os.getcwd(), 'raw_data')

# list all vhdr files in eeg folder
vhdr_files = [f for f in os.listdir(eeg_path) if f.endswith('.vhdr')]
vhdr_files.sort()
print(vhdr_files)

# Select one of the study subjects (=vhdr_files)
filename = vhdr_files[0]

# List known noisy channels here
noisy_channels = []

# Get data drom the opinion test
response_path = os.path.join(os.getcwd(), 'keyboard_opinion')
response_file = [f for f in os.listdir(response_path) if f.startswith(filename[:6])]
response = response_file[0]
filename, response

In [ ]:
# Read raw data, set montage and plot. Also click bad channels.

raw = mne.io.read_raw_brainvision(os.path.join(eeg_path, filename), stim_channel=True, preload=True, eog = ('EOG1', 'EOG2','EOG3','EOG4'))
print('Data downloaded')

montage = mne.channels.read_montage("standard_1020")
raw.set_montage(montage)
print('Montage set')

raw.plot()

In [ ]:
# Crop the data

events = mne.find_events(raw, stim_channel = 'STI 014')
event_times = events[:,0]

raw1 = raw.copy().crop((event_times[0]/1000)-20, (event_times[19]/1000)+10)
raw2 = raw.copy().crop((event_times[20]/1000)-10, (event_times[39]/1000)+10)
raw3 = raw.copy().crop((event_times[40]/1000)-10, (event_times[59]/1000)+10)
raw4 = raw.copy().crop((event_times[60]/1000)-10, (event_times[79]/1000)+10)
raw5 = raw.copy().crop((event_times[80]/1000)-10, (event_times[99]/1000)+10)
raw6 = raw.copy().crop((event_times[100]/1000)-10, (event_times[119]/1000)+10)

raw1.append([raw2, raw3, raw4, raw5, raw6])
raw = raw1
del raw1, raw2, raw3, raw4, raw5, raw6

raw.plot()

# Filter the data
raw.filter(1, 40., fir_design='firwin')

# Interpolate bad channels
raw = raw.interpolate_bads(reset_bads=True)

In [ ]:
# FIT ICA

# Set the parameters
n_components = 25
method = 'fastica'
decim = 3 
random_state = 23

# Run the analysis
ica = ICA(n_components=n_components, method=method, random_state=random_state)
reject = None
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False)

ica.fit(raw, picks=picks, decim=decim, reject=reject)

In [ ]:
# Search EOG artifacts and exclude those

eog_average = create_eog_epochs(raw, reject=reject, picks=picks).average()
eog_epochs = create_eog_epochs(raw, reject=reject)  # get single EOG trials
eog_inds, scores = ica.find_bads_eog(eog_epochs, threshold = 2) # find via correlation

# Plot ICA
ica.plot_scores(scores, exclude=eog_inds)        # look at r scores of components
ica.plot_sources(eog_average, exclude=eog_inds)  # look at source time course
ica.plot_overlay(eog_average, exclude=eog_inds, show=False)
                    
ica.exclude.extend(eog_inds)
del eog_inds, scores, eog_epochs, eog_average

In [ ]:
# Study rest of the components manually
ica.plot_components(inst = raw)

In [ ]:
# Excluding buffer. PLEASE MAKE EDITS HERE BEFORE RUNNING ica.exclude.extend(exclude_list)
exclude_list = [] # List bad components here

In [ ]:
# Exclude other bad components here and plot all the components once again
ica.exclude.extend(exclude_list)
ica.plot_components(inst = raw)

In [ ]:
# Save ICA to folder ICA.

if filename[6] == '.':
    ica.save('ICA/' + filename[:6] + '-ica.fif')
    print('ICA solution saved for subject: ' + filename[:6])
else:
    ica.save('ICA/' + filename[:8] + '-ica.fif')
    print('ICA solution saved for subject: ' + filename[:8])

In [ ]:
# If needed download ICA from folder ICA
if filename[6] == '.':
    ica = mne.preprocessing.read_ica('ICA/' + filename[:6] + '-ica.fif')
else:
    ica = mne.preprocessing.read_ica('ICA/' + filename[:8] + '-ica.fif')


In [ ]:
# Apply ICA to cropped, filtered and interpolated data

ica.apply(raw)
ica.plot_overlay(raw)

In [ ]:
# See how preprossing went
raw.plot()

In [ ]:
# Save preprocessed data to folder preprocessed
if filename[6] == '.':
    raw.save('preprocessed/' + filename[:6] + '-raw.fif', overwrite = True)
    print('Preprocessed data saved for subject: ' + filename[:6])
else:
    raw.save('preprocessed/' + filename[:8] + '-raw.fif', overwrite = True)
    print('Preprocessed data saved for subject: ' + filename[:8])

All measurements should be now processed and saved to preprocessed-folder before moving any further

In [ ]:
# This loop processes all the measurements at once

for filename in vhdr_files:
    
    ## Find response path and keyboard_opinion files fron opinion test
    response_path = os.path.join(os.getcwd(), 'keyboard_opinion')
    response_file = [f for f in os.listdir(response_path) if f.startswith(filename[:6])]
    response = response_file[0]


    df = pd.read_csv('keyboard_opinion/' + response)
    #print(df.head())

    # Save disagrees and neutrals to lists
    if filename[6] == '.':
        disagrees = list(df['disagree_en'])
        neutrals = list(df['neutral_en'])
    else:
        disagrees = list(df['disagree_fin'])
        neutrals = list(df['neutral_fin'])

        
    # Read unknown brands
    df = pd.read_csv('unknown/unknown_dataframe.csv')
    #print(df.head())

    unknow = list(df[str(filename[:6])])
    unknow = [x for x in unknow if ~np.isnan(x)] # Drop NaNs
    if filename[6] == '.':
        unknow.extend(list(df[str(filename[:6]) + '_ad']))
    unknow = [x for x in unknow if ~np.isnan(x)] # Drop NaNs

    # First sublist contains dislikes and second favourite
    # First element (list) at dislike/favourite sublist is matching and second conflicting
    # If value is NaN there is no values
    # Neutrals are skipped
    
    # IDs, e.g. 990291 presents that subjects English test and 990291_F Finnish test
    favourite_dict = { '990291': [[[125],[185]],[[np.nan],[np.nan]]], '990291_F': [[[np.nan],[np.nan]],[[np.nan],[np.nan]]], 
                      '928376': [[[np.nan],[np.nan]],[[156, 158],[216, 218]]], '928376_F': [[[np.nan],[np.nan]],[[156, 158],[216, 218]]], 
                      '884723': [[[125, 185],[np.nan]],[[np.nan],[np.nan]]], '884723_F': [[[125],[185]],[[np.nan],[np.nan]]], 
                      '554432': [[[145],[206]],[[19, 27, 173],[79, 87, 233]]], '554432_F': [[[145],[206]],[[19, 27],[79, 87]]],
                      '477819': [[[np.nan],[np.nan]],[[145, 151],[205, 211]]], '477819_F': [[[np.nan],[np.nan]],[[145, 151, 205],[211]]],
                      '382733': [[[121],[181]],[[7],[67]]], '382733_F': [[[np.nan],[121, 181]],[[7],[67]]]
                    }

    if filename[6] == '.':
        ID_dict = filename[:6] + '_F'
    else:
        ID_dict = filename[:6]

    temp_list = favourite_dict[ID_dict]

    dislike_match = temp_list[0][0]
    dislike_conf = temp_list[0][1]
    favo_match = temp_list[1][0]
    favo_conf = temp_list[1][1]


    # Find and read preprocessed data
    if filename[6] == '.':
        data_file = 'preprocessed/' + filename[:6] +'-raw.fif'
    else:
        data_file = 'preprocessed/' + filename[:8] +'-raw.fif'
    raw = mne.io.read_raw_fif(data_file, preload=True)

    # Give event_IDs to groups (stimuli uses event_IDs 1-240)
    event_id = {'Finnish Match': 241, 'Finnish Conf': 242,
                'Foreing Match': 243, 'Foreing Conf': 244, 
                'Neutrals': 245, 'Favourite Match': 246,
                'Favourite Conf': 247, 'Dislike Match': 248,
                'Dislike Conf': 249, 'Unknowns': 250}

    # Set parameters for epochs
    tmin, tmax = -0.2, 0.7
    baseline = (-0.2, 0)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False)   

    
    # Help function for event grouping
    def check_if_in(value, answers):
        if value in answers:
            return True
        else:
            return False

    # Search events
    events = mne.find_events(raw, stim_channel='STI 014')

    
    # Process events
    i = 0
    for value in events[:, 2]:
        if value in unknow: # Exclude unknown brands
            events[i, 2] = 250
        elif value in favo_match:
            events[i, 2] = 246
        elif value in favo_conf:
            events[i, 2] = 247
        elif value in dislike_match:
            events[i, 2] = 248
        elif value in dislike_conf:
            events[i, 2] = 249
        elif value <=60:    # Finnish matchings
            events[i, 2] = 242 if check_if_in(value, disagrees) else 245 if check_if_in(value, neutrals) else 241
        elif value <=120: # Finnish conflicting
            events[i, 2] = 241 if check_if_in(value, disagrees) else 245 if check_if_in(value, neutrals) else 242
        elif value <=180: # Foreing matchings
            events[i, 2] = 244 if check_if_in(value, disagrees) else 245 if check_if_in(value, neutrals) else 243
        elif 180 < value and value <=240: # Foreing conflicting
            events[i, 2] = 243 if check_if_in(value, disagrees) else 245 if check_if_in(value, neutrals) else 244
        else:
            raise ValueError('WRONG TRIGGER VALUE')    
        i += 1

    #print(events)
    if str(favo_match[0]) == 'nan':
        del event_id['Favourite Match']
        print('No favourite matching stimuli')
    if str(favo_conf[0]) == 'nan':
        del event_id['Favourite Conf']
        print('No favourite conflicting stimuli')
    if str(dislike_match[0]) == 'nan':
        del event_id['Dislike Match']
        print('No dislike matching stimuli')
    if str(dislike_conf[0]) == 'nan':
        del event_id['Dislike Conf']
        print('No dislike conflicting stimuli')

    # Make epochs
    epochs = mne.Epochs(raw, events, event_id, tmin= tmin, tmax = tmax, preload= True, baseline=baseline, picks=picks)
    print('Number of events:', len(events))
    print('Unique event codes:', np.unique(events[:, 2]))

    # Save epochs to folder processed_epochs
    if filename[6] == '.':
        epochs.save('processed_epochs/' + filename[:6] + '-epo.fif')
        print('Epoch data saved for subject: ' + filename[:6])
    else:
        epochs.save('processed_epochs/' + filename[:8] + '-epo.fif')
        print('Epoch data saved for subject: ' + filename[:8])

In [ ]:
# If needed read epochs

for filename in vhdr_files:
    if filename[6] == '.':
        epochs = mne.read_epochs('processed_epochs/' + filename[:6] + '-epo.fif')

    else:
        epochs = mne.read_epochs('processed_epochs/' + filename[:8] + '-epo.fif')
    print(epochs)